# Visualize Exported Data

This notebook can be used to visualize data exported from the camera network GUI to the SHDOM reconstruction algorithm.

In [1]:
%matplotlib inline
import cPickle
import matplotlib.pyplot as plt
import mayavi.mlab as mlab
import numpy as np
import os
import scipy.io as sio
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

## Load the Exported Data

In [2]:
BASE_PATH = r"..\..\scripts_client\reconstruction\2017_04_22_15_30_00"
with open(os.path.join(BASE_PATH, "export_data.pkl"), "rb") as f:
    data = cPickle.load(f)
grid = sio.loadmat(os.path.join(BASE_PATH, "grid.mat"))

In [3]:
X, Y, Z = np.meshgrid(grid["X"], grid["Y"], grid["Z"])

## Show the Visibility Grid

In [4]:
visibility = 0 
for k in data.keys():
    visibility +=  data[k]["Visibility"]
vis = np.zeros_like(visibility)
#vis = visibility
vis[visibility > 2] = 1

In [5]:
s = mlab.pipeline.scalar_field(X, Y, Z, vis)
s.spacing = [1, 1, 1]
s.update_image_data = True    
mlab.pipeline.image_plane_widget(s,
                            plane_orientation='x_axes',
                            slice_index=10,
                        )
mlab.pipeline.image_plane_widget(s,
                            plane_orientation='y_axes',
                            slice_index=10,
                        )
mlab.outline()
mlab.show()

## Visualize the Exported Images and Masks

In [6]:
ids = sorted(data.keys())

@interact(ind=(0, 9), intensity=(1, 100))
def plot(ind, intensity):
    f, axes = plt.subplots(ncols=3, figsize=(8, 24))
    cam = data[ids[ind]]
    img = np.dstack((cam["R"], cam["G"], cam["B"]))
    img = np.clip(img/intensity, 0, 1)
    axes[0].imshow(img, origin="bottom", interpolation="nearest")
    axes[1].imshow(cam["MASK"], origin="bottom", interpolation="nearest")
    axes[2].imshow(cam["SUN_MASK"], origin="bottom", interpolation="nearest")
    plt.show()

## Visualize Ray Directions

In [7]:
ids = sorted(data.keys())

@interact(ind=(0, 9))
def plot(ind):
    f, axes = plt.subplots(ncols=2, figsize=(8, 24))
    cam = data[ids[ind]]
    axes[0].imshow(cam["PHI"], origin="bottom")
    axes[0].set_title("PHI")
    axes[1].imshow(cam["PSI"], origin="bottom")
    axes[1].set_title("PSI")
    plt.show()